# Importamos datos

In [1]:
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer, KNNImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import category_encoders as ce # Para Target Encoding si se usa
import itertools # Para generar combinaciones de características
import warnings

# Ignorar warnings para mantener la salida limpia
warnings.filterwarnings('ignore')

# Cargar datos
print("Cargando datos train.csv y test.csv...")
try:
    df_entrenamiento_bruto = pd.read_csv('train.csv')
    df_prueba_bruto = pd.read_csv('test.csv')
    print("Datos cargados correctamente.")
    print("\nPrimeras 5 filas de df_entrenamiento_bruto:")
    print(df_entrenamiento_bruto.head())
except FileNotFoundError:
    print("ERROR: Asegúrate de que 'train.csv' y 'test.csv' estén en el directorio correcto.")
    df_entrenamiento_bruto = pd.DataFrame() 
    df_prueba_bruto = pd.DataFrame()

Cargando datos train.csv y test.csv...
Datos cargados correctamente.

Primeras 5 filas de df_entrenamiento_bruto:
      id            timestamp modo_operacion operador  temperatura    presion  \
0  15000  2021-04-24 03:00:00  mantenimiento      NaN    20.428941  58.474497   
1  15001  2020-09-26 19:00:00  mantenimiento        E     1.332931  84.784184   
2  15002  2021-07-02 15:00:00           auto        D    17.501579  65.057682   
3  15003  2020-02-05 22:00:00           auto      NaN     8.118801  70.127615   
4  15004  2020-09-05 18:00:00         manual      NaN    -0.353122  90.738090   

   sensor_ruido   sensor_3  fallo  
0      0.349400  18.301299      0  
1     -2.252684   7.295245      0  
2      0.359853  34.271305      0  
3      0.489166  -3.748938      0  
4     -1.505866  10.157637      0  


In [2]:
# --- Variables Globales y Listas de Características ---
objetivo = 'fallo'

# Características originales
columnas_numericas_base = ['temperatura', 'presion', 'sensor_ruido', 'sensor_3']
columnas_categoricas = ['modo_operacion', 'operador'] # Renombrado desde _base para claridad

# Columnas a procesar especialmente
columna_id = 'id'
columna_timestamp = 'timestamp'

print("--- Variables Globales Definidas ---")
print(f"  Objetivo: {objetivo}")
print(f"  Características numéricas base: {columnas_numericas_base}")
print(f"  Características categóricas: {columnas_categoricas}")
print(f"  Columna ID a eliminar: {columna_id}")
print(f"  Columna Timestamp a procesar: {columna_timestamp}")

# Mostrar NaNs iniciales
if not df_entrenamiento_bruto.empty:
    print("\nNaNs en df_entrenamiento_bruto:")
    print(df_entrenamiento_bruto.isnull().sum())
if not df_prueba_bruto.empty:
    print("\nNaNs en df_prueba_bruto:")
    print(df_prueba_bruto.isnull().sum())

--- Variables Globales Definidas ---
  Objetivo: fallo
  Características numéricas base: ['temperatura', 'presion', 'sensor_ruido', 'sensor_3']
  Características categóricas: ['modo_operacion', 'operador']
  Columna ID a eliminar: id
  Columna Timestamp a procesar: timestamp

NaNs en df_entrenamiento_bruto:
id                   0
timestamp            0
modo_operacion       0
operador          2731
temperatura          0
presion            625
sensor_ruido         0
sensor_3             0
fallo                0
dtype: int64

NaNs en df_prueba_bruto:
id                   0
timestamp            0
modo_operacion       0
operador          2700
temperatura          0
presion            625
sensor_ruido         0
sensor_3             0
dtype: int64


# Preparamos Datos

## Columnas id y TimeStamp

In [3]:
print("\n--- Iniciando Preprocesamiento ---")
if not df_entrenamiento_bruto.empty:
    # Crear copias de trabajo
    df_entrenamiento = df_entrenamiento_bruto.copy()
    df_prueba = df_prueba_bruto.copy()

    # --- 3.1. Manejar columna 'id' ---
    print(f"\nProcesando columna ID: '{columna_id}'...")
    if columna_id in df_entrenamiento.columns:
        print(f"  Eliminando '{columna_id}' de df_entrenamiento.")
        df_entrenamiento = df_entrenamiento.drop(columns=[columna_id])
    if columna_id in df_prueba.columns:
        print(f"  Eliminando '{columna_id}' de df_prueba.")
        df_prueba = df_prueba.drop(columns=[columna_id])

    # --- 3.2. Ingeniería de Características para 'timestamp' ---
    print(f"\nRealizando ingeniería de características para la columna: '{columna_timestamp}'...")
    def ingenieria_caracteristicas_tiempo(df, nombre_columna):
        if nombre_columna not in df.columns:
            print(f"  Advertencia: La columna '{nombre_columna}' no existe en el DataFrame. Omitiendo ingeniería de tiempo.")
            return df, []
            
        print(f"  Convirtiendo '{nombre_columna}' a datetime...")
        df[nombre_columna] = pd.to_datetime(df[nombre_columna])
        print(f"  Extrayendo características de tiempo de '{nombre_columna}'...")
        df['hora'] = df[nombre_columna].dt.hour
        df['dia_semana'] = df[nombre_columna].dt.dayofweek # Lunes=0, Domingo=6
        df['mes'] = df[nombre_columna].dt.month
        df['dia_del_anio'] = df[nombre_columna].dt.dayofyear # Renombrado para claridad
        df['es_fin_de_semana'] = (df[nombre_columna].dt.dayofweek >= 5).astype(int) # Renombrado
        
        caracteristicas_generadas = ['hora', 'dia_semana', 'mes', 'dia_del_anio', 'es_fin_de_semana']
        
        print(f"  Eliminando la columna original '{nombre_columna}'.")
        df = df.drop(columns=[nombre_columna])
        return df, caracteristicas_generadas

    df_entrenamiento, caracteristicas_tiempo_generadas_train = ingenieria_caracteristicas_tiempo(df_entrenamiento, columna_timestamp)
    df_prueba, _ = ingenieria_caracteristicas_tiempo(df_prueba, columna_timestamp) # No necesitamos la lista de test, asumimos que es la misma

    # Lista completa de características numéricas (incluyendo las de tiempo)
    # Asegurar que solo se añaden si se generaron (por si timestamp_column faltara)
    todas_columnas_numericas = columnas_numericas_base + caracteristicas_tiempo_generadas_train

    print(f"\nNuevas características de tiempo generadas: {caracteristicas_tiempo_generadas_train}")
    print(f"Todas las características numéricas para imputación serán: {todas_columnas_numericas}")
    print("\nPrimeras 5 filas de df_entrenamiento después de procesar ID y timestamp:")
    print(df_entrenamiento.head())
else:
    print("Error: df_entrenamiento_bruto no está definido o está vacío. Ejecuta las celdas anteriores.")
    # Crear DataFrames vacíos para evitar errores en celdas subsiguientes si se desea continuar
    df_entrenamiento = pd.DataFrame()
    df_prueba = pd.DataFrame()
    todas_columnas_numericas = columnas_numericas_base


--- Iniciando Preprocesamiento ---

Procesando columna ID: 'id'...
  Eliminando 'id' de df_entrenamiento.
  Eliminando 'id' de df_prueba.

Realizando ingeniería de características para la columna: 'timestamp'...
  Convirtiendo 'timestamp' a datetime...
  Extrayendo características de tiempo de 'timestamp'...
  Eliminando la columna original 'timestamp'.
  Convirtiendo 'timestamp' a datetime...
  Extrayendo características de tiempo de 'timestamp'...
  Eliminando la columna original 'timestamp'.

Nuevas características de tiempo generadas: ['hora', 'dia_semana', 'mes', 'dia_del_anio', 'es_fin_de_semana']
Todas las características numéricas para imputación serán: ['temperatura', 'presion', 'sensor_ruido', 'sensor_3', 'hora', 'dia_semana', 'mes', 'dia_del_anio', 'es_fin_de_semana']

Primeras 5 filas de df_entrenamiento después de procesar ID y timestamp:
  modo_operacion operador  temperatura    presion  sensor_ruido   sensor_3  \
0  mantenimiento      NaN    20.428941  58.474497      0.

## Quitar los Nan

### En las Numericas
Hay dos opciones: 

Imputar con la Mediana, se entrena a un modelo y se sustituye los Nan con la prediccion. Es buena pero no siempre fiable

Imputar con KNNImputer, una version mas avanzada de imputar con la mediana. El problema es que hay que escalar los datos antes de pasarlos por el Imputer

In [4]:
df_entrenamiento_procesado = df_entrenamiento.copy()
df_prueba_procesado = df_prueba.copy()

# Columnas numéricas existentes en los dataframes para imputar
columnas_numericas_a_imputar_train = [col for col in todas_columnas_numericas if col in df_entrenamiento_procesado.columns]
columnas_numericas_a_imputar_test = [col for col in todas_columnas_numericas if col in df_prueba_procesado.columns]

print("  Usando Imputación por Mediana...")
if columnas_numericas_a_imputar_train:
    imputador_mediana = SimpleImputer(strategy='median')
    df_entrenamiento_procesado[columnas_numericas_a_imputar_train] = imputador_mediana.fit_transform(df_entrenamiento_procesado[columnas_numericas_a_imputar_train])
    if columnas_numericas_a_imputar_test:
        # Asegurar que test tenga las mismas columnas numéricas que se ajustaron en train
        cols_comunes_test = [col for col in columnas_numericas_a_imputar_test if col in imputador_mediana.feature_names_in_]
        if cols_comunes_test:
            df_prueba_procesado[cols_comunes_test] = imputador_mediana.transform(df_prueba_procesado[cols_comunes_test])
else:
    print("  No hay columnas numéricas para imputar con Mediana en el set de entrenamiento.")

  Usando Imputación por Mediana...


In [5]:
escalador_knn = StandardScaler() 

# Ajustar y transformar train
df_entrenamiento_escalado_num = escalador_knn.fit_transform(df_entrenamiento_procesado[columnas_numericas_a_imputar_train])

imputador_knn = KNNImputer(n_neighbors=5) 
df_entrenamiento_imputado_escalado = imputador_knn.fit_transform(df_entrenamiento_escalado_num) # Fit y transform en train escalado

# Desescalar train
df_entrenamiento_procesado[columnas_numericas_a_imputar_train] = escalador_knn.inverse_transform(df_entrenamiento_imputado_escalado)

# Transformar test
if columnas_numericas_a_imputar_test:
    # Asegurar que test tenga las mismas columnas numéricas que se ajustaron/escalaron en train
    cols_comunes_test_knn = [col for col in columnas_numericas_a_imputar_test if col in escalador_knn.feature_names_in_]
    if cols_comunes_test_knn:
        df_prueba_escalado_num = escalador_knn.transform(df_prueba_procesado[cols_comunes_test_knn])
        df_prueba_imputado_escalado = imputador_knn.transform(df_prueba_escalado_num) # Solo transform en test escalado
        df_prueba_procesado[cols_comunes_test_knn] = escalador_knn.inverse_transform(df_prueba_imputado_escalado)
    else:
        print("    Advertencia: No hay columnas numéricas coincidentes en el test set para KNNImputer después del escalado de train.")

### En las categóricas
Aqui simplemente reemplazamos los Nan por un nuevo valor llamado "Desconocido"

In [7]:
columnas_categoricas_a_imputar_train = [col for col in columnas_categoricas if col in df_entrenamiento_procesado.columns]
columnas_categoricas_a_imputar_test = [col for col in columnas_categoricas if col in df_prueba_procesado.columns]

imputador_constante_cat = SimpleImputer(strategy='constant', fill_value='Faltante') # Traducido "Missing"

# Ajustar y transformar train
df_entrenamiento_procesado[columnas_categoricas_a_imputar_train] = imputador_constante_cat.fit_transform(df_entrenamiento_procesado[columnas_categoricas_a_imputar_train])

# Transformar test
if columnas_categoricas_a_imputar_test:
    # Asegurar que test tenga las mismas columnas categóricas que se ajustaron en train
    cols_comunes_test_cat = [col for col in columnas_categoricas_a_imputar_test if col in imputador_constante_cat.feature_names_in_]
    if cols_comunes_test_cat:
            df_prueba_procesado[cols_comunes_test_cat] = imputador_constante_cat.transform(df_prueba_procesado[cols_comunes_test_cat])
print(f"  Imputando NaNs en columnas categóricas con '{imputador_constante_cat.fill_value}'.")


print("\nComprobación de NaNs después de toda la imputación:")
print("NaNs en df_entrenamiento_procesado (solo columnas con NaNs):")
print(df_entrenamiento_procesado.isnull().sum().loc[lambda x: x > 0])
print("\nNaNs en df_prueba_procesado (solo columnas con NaNs):")
print(df_prueba_procesado.isnull().sum().loc[lambda x: x > 0])

  Imputando NaNs en columnas categóricas con 'Faltante'.

Comprobación de NaNs después de toda la imputación:
NaNs en df_entrenamiento_procesado (solo columnas con NaNs):
Series([], dtype: int64)

NaNs en df_prueba_procesado (solo columnas con NaNs):
Series([], dtype: int64)


## Tratar las categoricas para los modelos
Vamos a probar dos, OneHot y Target Encoding.

In [8]:

# df_entrenamiento_final y df_prueba_final serán el resultado de esta celda
df_entrenamiento_final = df_entrenamiento_procesado.copy()
df_prueba_final = df_prueba_procesado.copy()

# Columnas categóricas existentes para la codificación
columnas_categoricas_a_codificar_train = [col for col in columnas_categoricas if col in df_entrenamiento_final.columns]
columnas_categoricas_a_codificar_test = [col for col in columnas_categoricas if col in df_prueba_final.columns]


In [9]:
codificador_ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore', drop='first')

# Ajustar y transformar train
matriz_codificada_entrenamiento = codificador_ohe.fit_transform(df_entrenamiento_final[columnas_categoricas_a_codificar_train])
df_codificado_entrenamiento = pd.DataFrame(matriz_codificada_entrenamiento, 
                                            columns=codificador_ohe.get_feature_names_out(columnas_categoricas_a_codificar_train), 
                                            index=df_entrenamiento_final.index)

# Eliminar categóricas originales y unir las codificadas
df_entrenamiento_final = df_entrenamiento_final.drop(columns=columnas_categoricas_a_codificar_train)
df_entrenamiento_final = df_entrenamiento_final.join(df_codificado_entrenamiento)


matriz_codificada_prueba = codificador_ohe.transform(df_prueba_final[columnas_categoricas_a_codificar_test])
df_codificado_prueba = pd.DataFrame(matriz_codificada_prueba, 
                                    columns=codificador_ohe.get_feature_names_out(columnas_categoricas_a_codificar_test), 
                                    index=df_prueba_final.index)
df_prueba_final = df_prueba_final.drop(columns=columnas_categoricas_a_codificar_test)
df_prueba_final = df_prueba_final.join(df_codificado_prueba)


In [44]:
codificador_objetivo = ce.TargetEncoder(cols=columnas_categoricas_a_codificar_train, smoothing=10.0)

# Ajustar y transformar train (modifica el DataFrame directamente)
df_entrenamiento_final[columnas_categoricas_a_codificar_train] = codificador_objetivo.fit_transform(df_entrenamiento_final[columnas_categoricas_a_codificar_train], df_entrenamiento_final[objetivo])

# Transformar test
df_prueba_final[columnas_categoricas_a_codificar_test] = codificador_objetivo.transform(df_prueba_final[columnas_categoricas_a_codificar_test])

In [10]:
print(f"\nForma de df_entrenamiento_final (después de codificación categórica): {df_entrenamiento_final.shape if 'df_entrenamiento_final' in locals() else 'No definido'}")
if 'df_entrenamiento_final' in locals() and not df_entrenamiento_final.empty:
    print("Tipos de datos en df_entrenamiento_final:")
    print(df_entrenamiento_final.dtypes)
    print("\nVerificación final de NaNs en df_entrenamiento_final (deberían ser 0):")
    print(df_entrenamiento_final.isnull().sum().loc[lambda x: x > 0])


Forma de df_entrenamiento_final (después de codificación categórica): (15000, 17)
Tipos de datos en df_entrenamiento_final:
temperatura                     float64
presion                         float64
sensor_ruido                    float64
sensor_3                        float64
fallo                             int64
hora                            float64
dia_semana                      float64
mes                             float64
dia_del_anio                    float64
es_fin_de_semana                float64
modo_operacion_mantenimiento    float64
modo_operacion_manual           float64
operador_C                      float64
operador_D                      float64
operador_E                      float64
operador_F                      float64
operador_Faltante               float64
dtype: object

Verificación final de NaNs en df_entrenamiento_final (deberían ser 0):
Series([], dtype: int64)


## Datos sin Tocar

In [11]:
df_entrenamiento_base = df_entrenamiento_final.copy()
df_prueba_base = df_prueba_final.copy()



## Datos quitando las no prometedoras (Sensor Ruido)

In [12]:
df_entrenamiento_reducido = df_entrenamiento_final.copy()
df_prueba_reducido = df_prueba_final.copy()
columna_a_quitar = 'sensor_ruido'

if columna_a_quitar in df_entrenamiento_reducido.columns:
    df_entrenamiento_reducido = df_entrenamiento_reducido.drop(columns=[columna_a_quitar])
else:
    print(f"  Advertencia (Reducido - Train): Columna '{columna_a_quitar}' no encontrada para eliminar.")
if columna_a_quitar in df_prueba_reducido.columns:
    df_prueba_reducido = df_prueba_reducido.drop(columns=[columna_a_quitar])
else:
    print(f"  Advertencia (Reducido - Test): Columna '{columna_a_quitar}' no encontrada para eliminar.")


## Datos relacionados TODOS con TODOS

In [13]:
df_entrenamiento_interacciones = df_entrenamiento_final.copy()
df_prueba_interacciones = df_prueba_final.copy()


for col1, col2 in itertools.combinations(columnas_numericas_base, 2):
    if col1 in df_entrenamiento_interacciones.columns and col2 in df_entrenamiento_interacciones.columns:
        nombre_col_interaccion = f"{col1}_x_{col2}" # Renombrado
        df_entrenamiento_interacciones[nombre_col_interaccion] = df_entrenamiento_interacciones[col1] * df_entrenamiento_interacciones[col2]
        
        if col1 in df_prueba_interacciones.columns and col2 in df_prueba_interacciones.columns:
                df_prueba_interacciones[nombre_col_interaccion] = df_prueba_interacciones[col1] * df_prueba_interacciones[col2]
        else:
                print(f"    Advertencia (Interacciones - Test): '{col1}' o '{col2}' no encontradas para crear '{nombre_col_interaccion}'. Se rellenará con 0.")
                df_prueba_interacciones[nombre_col_interaccion] = 0 # Imputar con 0 si falta en test
    else:
        print(f"    Advertencia (Interacciones - Train): '{col1}' o '{col2}' no encontradas para crear interacción. Se omite.")
        

# Probar modelos:

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, f1_score

In [15]:
# --- Bucle de Entrenamiento y Evaluación de Modelos ---
print("\n--- Iniciando Evaluación de Modelos ---")

conjuntos_datos_a_evaluar = []

if 'df_entrenamiento_base' in locals() and not df_entrenamiento_base.empty:
    conjuntos_datos_a_evaluar.append({'nombre': 'Base', 'datos': df_entrenamiento_base})
else:
    print("Advertencia: 'df_entrenamiento_base' no está definido o está vacío. Se omitirá.")

if 'df_entrenamiento_reducido' in locals() and not df_entrenamiento_reducido.empty:
    conjuntos_datos_a_evaluar.append({'nombre': 'Caracteristicas Reducidas', 'datos': df_entrenamiento_reducido})
else:
    print("Advertencia: 'df_entrenamiento_reducido' no está definido o está vacío. Se omitirá.")

if 'df_entrenamiento_interacciones' in locals() and not df_entrenamiento_interacciones.empty:
    conjuntos_datos_a_evaluar.append({'nombre': 'Interaccion de Caracteristicas', 'datos': df_entrenamiento_interacciones})
else:
    print("Advertencia: 'df_entrenamiento_interacciones' no está definido o está vacío. Se omitirá.")

if not conjuntos_datos_a_evaluar:
    print("ERROR CRÍTICO: No hay DataFrames válidos para evaluar. Deteniendo el script.")
else:
    # Preparar lista de modelos
    modelos_a_evaluar = [
        ('Árbol de Decisión', DecisionTreeClassifier(random_state=42)), # Traducido
        ('Naive Bayes Gaussiano', GaussianNB()), # Traducido
        ('Random Forest', RandomForestClassifier(random_state=42, n_jobs=-1)),
        ('Gradient Boosting', GradientBoostingClassifier(random_state=42)),
        ('Bagging', BaggingClassifier(random_state=42, n_jobs=-1))
    ]

    # Definir StratifiedKFold y el F1 scorer
    # Asumimos que la clase positiva para F1-score es '1'.
    evaluador_f1 = make_scorer(f1_score, pos_label=1, zero_division=0)
    num_divisiones = 4 # Renombrado
    kfold_estratificado = StratifiedKFold(n_splits=num_divisiones, shuffle=True, random_state=42) # Renombrado

    resultados_completos = {} 
    resumen_f1_promedio = {} 

    for info_conjunto in conjuntos_datos_a_evaluar: # Renombrado
        nombre_conjunto = info_conjunto['nombre'] # Renombrado
        df_actual_entrenamiento = info_conjunto['datos'] # Renombrado
        
        print(f"\n======================================================================")
        print(f"--- Evaluando Conjunto de Datos: {nombre_conjunto} (Forma: {df_actual_entrenamiento.shape}) ---")
        print(f"======================================================================")

        if objetivo not in df_actual_entrenamiento.columns:
            print(f"  ERROR: La columna objetivo '{objetivo}' no se encuentra en '{nombre_conjunto}'. Omitiendo.")
            continue

        X_entrenamiento = df_actual_entrenamiento.drop(columns=[objetivo]) # Renombrado
        y_entrenamiento = df_actual_entrenamiento[objetivo] # Renombrado
        
        # Verificar columna objetivo
        valores_unicos_objetivo = y_entrenamiento.unique() # Renombrado
        if not pd.api.types.is_numeric_dtype(y_entrenamiento) or not all(val in [0, 1] for val in valores_unicos_objetivo if pd.notna(val)) or len(valores_unicos_objetivo) > 2 :
            print(f"  ADVERTENCIA: Columna objetivo '{objetivo}' en '{nombre_conjunto}' (Valores: {valores_unicos_objetivo[:5]}) no es binaria (0/1) o contiene NaNs.")
        
        resumen_f1_promedio[nombre_conjunto] = {}
        resultados_completos[nombre_conjunto] = {}

        for nombre_modelo, instancia_modelo in modelos_a_evaluar: # Renombrado
            print(f"\n  --- Modelo: {nombre_modelo} ---")
            if X_entrenamiento.empty:
                print(f"    Omitiendo {nombre_modelo} porque X_entrenamiento está vacío.")
                resumen_f1_promedio[nombre_conjunto][nombre_modelo] = np.nan
                resultados_completos[nombre_conjunto][nombre_modelo] = {'f1_promedio': np.nan, 'desv_est_f1': np.nan, 'f1_por_fold': [], 'error': 'X_entrenamiento vacío'}
                continue
            
            try:
                puntuaciones_cv_f1 = cross_val_score(instancia_modelo, X_entrenamiento, y_entrenamiento, cv=kfold_estratificado, scoring=evaluador_f1, n_jobs=-1) # Renombrado
                
                f1_promedio_modelo = np.mean(puntuaciones_cv_f1) # Renombrado
                desv_est_f1_modelo = np.std(puntuaciones_cv_f1) # Renombrado
                
                resumen_f1_promedio[nombre_conjunto][nombre_modelo] = f1_promedio_modelo
                resultados_completos[nombre_conjunto][nombre_modelo] = {'f1_promedio': f1_promedio_modelo, 'desv_est_f1': desv_est_f1_modelo, 'f1_por_fold': puntuaciones_cv_f1.tolist()}
                
                print(f"    F1-Score Promedio ({num_divisiones} folds): {f1_promedio_modelo:.4f} (std: {desv_est_f1_modelo:.4f})")
                print(f"    F1-Scores por fold: {[round(s, 4) for s in puntuaciones_cv_f1]}")
                
            except Exception as e:
                print(f"    ERROR al entrenar/evaluar {nombre_modelo} en {nombre_conjunto}: {e}")
                resumen_f1_promedio[nombre_conjunto][nombre_modelo] = np.nan
                resultados_completos[nombre_conjunto][nombre_modelo] = {'f1_promedio': np.nan, 'desv_est_f1': np.nan, 'f1_por_fold': [], 'error': str(e)}

    # Imprimir tabla resumen
    print(f"\n\n======================================================================")
    print(f"--- Tabla Resumen de F1-Score Promedio (clase positiva=1) ---")
    print(f"======================================================================")

    if resumen_f1_promedio:
        df_resumen = pd.DataFrame(resumen_f1_promedio).T # Renombrado
        orden_modelos = [name for name, _ in modelos_a_evaluar] # Renombrado
        columnas_existentes_modelos = [m for m in orden_modelos if m in df_resumen.columns] # Renombrado
        if columnas_existentes_modelos:
            df_resumen = df_resumen[columnas_existentes_modelos]
        
        print(df_resumen.to_string(float_format="%.4f"))
    else:
        print("No se generaron resultados para el resumen.")

print("\n¡Proceso de evaluación de modelos completado!")


--- Iniciando Evaluación de Modelos ---

--- Evaluando Conjunto de Datos: Base (Forma: (15000, 17)) ---

  --- Modelo: Árbol de Decisión ---
    F1-Score Promedio (4 folds): 0.8996 (std: 0.0039)
    F1-Scores por fold: [np.float64(0.8947), np.float64(0.901), np.float64(0.8974), np.float64(0.9052)]

  --- Modelo: Naive Bayes Gaussiano ---
    F1-Score Promedio (4 folds): 0.7225 (std: 0.0093)
    F1-Scores por fold: [np.float64(0.7141), np.float64(0.7325), np.float64(0.7123), np.float64(0.7311)]

  --- Modelo: Random Forest ---
    F1-Score Promedio (4 folds): 0.9147 (std: 0.0059)
    F1-Scores por fold: [np.float64(0.9071), np.float64(0.9161), np.float64(0.9121), np.float64(0.9233)]

  --- Modelo: Gradient Boosting ---
    F1-Score Promedio (4 folds): 0.9154 (std: 0.0063)
    F1-Scores por fold: [np.float64(0.9048), np.float64(0.9189), np.float64(0.9208), np.float64(0.9173)]

  --- Modelo: Bagging ---
    F1-Score Promedio (4 folds): 0.9135 (std: 0.0036)
    F1-Scores por fold: [np.flo

## Funcion Predict Modelo
Esta funcion, dado un modelo y una df hace un predict y lo guarda en su correspondiente csv

In [25]:
def predictCSV(model, model_name, df_toPredict, df_name):
    columns = ['id', 'fallo']
    csv_file = f"CSVModelos/{model_name}_{df_name}.csv"
    header = True

    for i in range(0, len(df_toPredict), 500):
        test_predict = model.predict(df_toPredict[i:i+500])
        
        indi = [j for j in range(i, i+500,1)]
        valor = [test_predict[k] for k in range(500)]

        results = pd.DataFrame(list(zip(indi,valor)), columns=columns)

        results.to_csv(csv_file, mode='a', header=header, index=False)
        header = False

## DECISIONTREE


In [28]:
def predictArbolDecision():
    df_to_train = df_entrenamiento_reducido.copy()
    df_toTest = df_prueba_reducido.copy()

    X_entrenamiento = df_to_train.drop(columns=[objetivo]) 
    y_entrenamiento = df_to_train[objetivo] 

    model = DecisionTreeClassifier(random_state=42)
    model.fit(X_entrenamiento,y_entrenamiento)


    predictCSV(model,"ArbolDecision",df_toTest,"Reducido")

## NAIVEBAYES


In [26]:
def predictNaiveBayes():
    df_to_train = df_entrenamiento_base.copy()
    df_toTest = df_prueba_base.copy()

    X_entrenamiento = df_to_train.drop(columns=[objetivo]) 
    y_entrenamiento = df_to_train[objetivo] 

    model = GaussianNB()
    model.fit(X_entrenamiento,y_entrenamiento)


    predictCSV(model,"NaiveBayes",df_toTest,"Basico")


## RANDOMFOREST


In [29]:
def predictRandomForest():
    df_to_train = df_entrenamiento_reducido.copy()
    df_toTest = df_prueba_reducido.copy()

    X_entrenamiento = df_to_train.drop(columns=[objetivo]) 
    y_entrenamiento = df_to_train[objetivo] 

    model = RandomForestClassifier(random_state=42, n_jobs=-1)
    model.fit(X_entrenamiento,y_entrenamiento)


    predictCSV(model,"RandomForest",df_toTest,"Reducido")

## GRADIENTBOOSTING

In [30]:

def predictGradientBoosting():
    df_to_train = df_entrenamiento_base.copy()
    df_toTest = df_prueba_base.copy()

    X_entrenamiento = df_to_train.drop(columns=[objetivo]) 
    y_entrenamiento = df_to_train[objetivo] 

    model = GradientBoostingClassifier(random_state=42)
    model.fit(X_entrenamiento,y_entrenamiento)


    predictCSV(model,"GradientBoosting",df_toTest,"Basico")


## BAGGING

In [31]:
def predictBagging():
    df_to_train = df_entrenamiento_reducido.copy()
    df_toTest = df_prueba_reducido.copy()

    X_entrenamiento = df_to_train.drop(columns=[objetivo]) 
    y_entrenamiento = df_to_train[objetivo] 

    model = BaggingClassifier(random_state=42, n_jobs=-1)
    model.fit(X_entrenamiento,y_entrenamiento)


    predictCSV(model,"ArbolDecision",df_toTest,"Reducido")

In [ ]:
#predictArbolDecision()
#predictNaiveBayes()
#predictRandomForest()
#predictGradientBoosting()
#predictBagging()